In [1]:
import os
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Danh sách các hãng và URL tương ứng
brands = {
    "samsung": {
        "category": "Điện thoại Samsung",
        "url": "https://www.thegioididong.com/dtdd-samsung"
    },
    "iphone": {
        "category": "Điện thoại iPhone",
        "url": "https://www.thegioididong.com/dtdd-apple-iphone"
    },
    "oppo": {
        "category": "Điện thoại OPPO",
        "url": "https://www.thegioididong.com/dtdd-oppo"
    }
}

# Header giả lập trình duyệt
headers = {"User-Agent": "Mozilla/5.0"}

# Tạo thư mục images/
os.makedirs("images", exist_ok=True)

# Tạo file CSV
with open("products.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["category", "product_name", "price", "image", "source"])

    for brand_key, brand_info in brands.items():
        category = brand_info["category"]
        source_url = brand_info["url"]
        print(f"\n📦 Đang cào dữ liệu: {category}...")

        # Tạo thư mục con cho từng hãng
        brand_dir = os.path.join("images", brand_key)
        os.makedirs(brand_dir, exist_ok=True)

        try:
            response = requests.get(source_url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")
            products = soup.select("ul.listproduct li")

            idx = 1
            for product in products:
                # Tên sản phẩm
                name_tag = product.select_one("h3")
                if not name_tag:
                    continue
                name = name_tag.get_text(strip=True)

                # Giá sản phẩm
                price_tag = product.select_one(".price strong") or product.select_one(".price")
                price = price_tag.get_text(strip=True) if price_tag else "N/A"

                # Ảnh
                img_tag = product.select_one("img")
                img_url = (
                    img_tag.get("data-src") or
                    img_tag.get("data-original") or
                    img_tag.get("src")
                ) if img_tag else None
                full_img_url = urljoin(source_url, img_url) if img_url else None

                image_path = ""
                if full_img_url and "http" in full_img_url:
                    try:
                        img_data = requests.get(full_img_url, headers=headers).content
                        image_path = os.path.join(brand_dir, f"{brand_key}_{idx}.jpg")
                        with open(image_path, "wb") as f:
                            f.write(img_data)
                        image_path = image_path.replace("\\", "/")  # Cho chuẩn đường dẫn
                    except Exception as e:
                        print(f"❌ Lỗi lưu ảnh {name}: {e}")
                        image_path = ""

                # Ghi dữ liệu vào CSV
                writer.writerow([category, name, price, image_path, source_url])
                print(f"✔️ {idx}. {name} | {price}")
                idx += 1

        except Exception as e:
            print(f"❌ Lỗi khi xử lý {category}: {e}")

print("\n✅ Hoàn tất! Dữ liệu đã được lưu trong 'products.csv' và thư mục 'images/' theo từng hãng.")



📦 Đang cào dữ liệu: Điện thoại Samsung...
✔️ 1. Samsung Galaxy Z Flip7 FE 5G 8GB/128GBMẫu mới | 22.990.000₫
✔️ 2. Samsung Galaxy Z Fold7 5G 12GB/256GBMẫu mới | 46.990.000₫
✔️ 3. Samsung Galaxy Z Flip7 5G 12GB/256GBMẫu mới | 28.990.000₫
✔️ 4. Samsung Galaxy A06 5G 6GB/128GB | 4.210.000₫
✔️ 5. Samsung Galaxy A36 5G 12GB/256GB | 9.610.000₫
✔️ 6. Samsung Galaxy S25 Ultra 5G 12GB/256GB | 28.380.000₫
✔️ 7. Samsung Galaxy Z Fold6 5G 12GB/256GB | 34.390.000₫
✔️ 8. Samsung Galaxy A26 5G 6GB/128GB | 6.270.000₫
✔️ 9. Samsung Galaxy S24 FE 5G 8GB/256GB | 14.060.000₫
✔️ 10. Samsung Galaxy S24 5G 8GB/256GB | 17.510.000₫
✔️ 11. Samsung Galaxy A56 5G 12GB/256GB | 11.280.000₫
✔️ 12. Samsung Galaxy S25 5G 12GB/256GB | 18.690.000₫
✔️ 13. Samsung Galaxy A35 5G 8GB/256GB | 7.830.000₫
✔️ 14. Samsung Galaxy A06 4GB/64GB | 2.940.000₫
✔️ 15. Samsung Galaxy A16 5G 8GB/256GB | 6.370.000₫
✔️ 16. Samsung Galaxy S24 Ultra 5G 12GB/256GB | 23.850.000₫
✔️ 17. Samsung Galaxy M35 5G 8GB/256GB | 7.830.000₫
✔️ 18. Samsun